In [1]:
!pip install neptune-notebooks > /dev/null # no output
!pip install neptune-client > /dev/null # no output

In [2]:
!conda install gdcm -c conda-forge -y

In [3]:
class CFG:
    debug=False
    image_size=360
    lr=8e-3
    batch_size=1
    epochs=200
    seed=2018
    N = 36
    n_fold=5

resume=False
fp16=False
accumulation_steps=10
accumulate=False
num_workers=4
quantiles = (0.2, 0.5, 0.8)
HM_SLICES = 40

In [4]:
%matplotlib inline

import numpy as np
import pandas as pd
import pydicom
import math
import matplotlib.pyplot as plt
import cv2
import gc
import random
import time
import os
from tqdm import tqdm
from multiprocessing import Pool
from contextlib import contextmanager
from pathlib import Path
from collections import defaultdict, Counter
from IPython.core.display import display, HTML
from time import perf_counter

from plotly import __version__
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
from plotly import figure_factory as FF
from plotly.graph_objs import *

from skimage import data
from skimage import measure, feature, morphology
from skimage.util import montage
from skimage.morphology import ball, disk, dilation, binary_erosion, remove_small_objects, erosion, closing, reconstruction, binary_closing, binary_dilation, binary_opening
from skimage.measure import label,regionprops, perimeter
#from skimage.filters import threshold_otsu, median
from skimage.filters import roberts, sobel
from skimage.segmentation import clear_border
from skimage.exposure import equalize_hist
from scipy import ndimage as ndi
from scipy.ndimage import binary_fill_holes
from scipy.stats import skew, kurtosis

import sklearn.metrics
from sklearn import preprocessing
from sklearn.metrics import accuracy_score, mean_absolute_error
from sklearn.model_selection import StratifiedKFold, GroupKFold, KFold, train_test_split, TimeSeriesSplit
from sklearn.utils import shuffle

from functools import partial

import torch
import torch.nn as nn
from torch.nn import init, Sequential
import torch.nn.functional as F
from torch.optim import Adam, SGD
from torch.optim.lr_scheduler import CosineAnnealingLR, ReduceLROnPlateau, StepLR
from torch.utils.data import DataLoader, Dataset
from torch.utils.data.sampler import SubsetRandomSampler, RandomSampler, SequentialSampler
from torch.autograd import Variable

import warnings 
warnings.filterwarnings('ignore')

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

INPUT_FOLDER = '../input/osic-pulmonary-fibrosis-progression/train/'

dicom_arrays_dir = Path('../input/dicom-arrays-processed/kaggle/dicom_arrays/')
os.makedirs(dicom_arrays_dir, exist_ok=True)

latent_dir = Path('/kaggle/features_dir/')
os.makedirs(latent_dir, exist_ok=True)

mask_dir = Path('/kaggle/masks/')
os.makedirs(mask_dir, exist_ok=True)

volume_array_file = Path('../input/data-stats/volume_array.pt')
kurts_array_file = Path('../input/data-stats/kurts_array.pt')
skews_array_file = Path('../input/data-stats/skews_array.pt')
means_array_file = Path('../input/data-stats/mean_array.pt')
stds_array_file = Path('../input/data-stats/std_array.pt')
medians_array_file = Path('../input/data-stats/median_array.pt')

model_dir = '../input/best-autoencoder-models'

MODELS = []
for filename in os.listdir(model_dir):
    if filename.endswith(".pt"): 
        print(os.path.join(model_dir, filename))
        MODELS.append(os.path.join(model_dir, filename))

#patients = os.listdir(INPUT_FOLDER)
#patients.sort()
init_notebook_mode(connected=True)

cpu
../input/best-autoencoder-models/model_fold_1.pt
../input/best-autoencoder-models/model_fold_0.pt
../input/best-autoencoder-models/model_fold_2.pt


In [ ]:
import neptune
neptune.init(api_token=os.getenv('NEPTUNE_API_TOKEN'),
             project_qualified_name=os.getenv('NEPTUNE_PROJECT'))

params={'epochs': CFG.epochs,
        'batch_size': CFG.batch_size,
        'lr': CFG.lr}

neptune.create_experiment(name='lstm-train', params=params)

In [7]:
# ====================================================
# Utils
# ====================================================

@contextmanager
def timer(name):
    t0 = time.time()
    LOGGER.info(f'[{name}] start')
    yield
    LOGGER.info(f'[{name}] done in {time.time() - t0:.0f} s.')

    
def init_logger(log_file='train.log'):
    from logging import getLogger, DEBUG, FileHandler,  Formatter,  StreamHandler
    
    log_format = '%(asctime)s %(levelname)s %(message)s'
    
    stream_handler = StreamHandler()
    stream_handler.setLevel(DEBUG)
    stream_handler.setFormatter(Formatter(log_format))
    
    file_handler = FileHandler(log_file)
    file_handler.setFormatter(Formatter(log_format))
    
    logger = getLogger('fibrosis')
    logger.setLevel(DEBUG)
    logger.addHandler(stream_handler)
    logger.addHandler(file_handler)
    
    return logger

LOG_FILE = 'train.log'
LOGGER = init_logger(LOG_FILE)


def seed_torch(seed=2020):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True

seed_torch(seed=CFG.seed)

In [8]:
patient_files = list(os.listdir(INPUT_FOLDER))
print("Number of folders:", len(patient_files))

Number of folders: 176


In [9]:
train_csv = pd.read_csv('../input/osic-pulmonary-fibrosis-progression/train.csv')
train_csv = train_csv.drop_duplicates(subset=['Patient', 'Weeks'])
test_csv = pd.read_csv('../input/osic-pulmonary-fibrosis-progression/test.csv')
sub_csv = pd.read_csv('../input/osic-pulmonary-fibrosis-progression/sample_submission.csv')

sub_csv['Weeks']   = sub_csv['Patient_Week'].apply( lambda x: int(x.split('_')[-1]) )
sub_csv['Patient'] = sub_csv['Patient_Week'].apply( lambda x: x.split('_')[0] ) 
sub_csv =  sub_csv[['Patient','Weeks','Confidence','Patient_Week']]
sub_csv = sub_csv.merge(test_csv.drop('Weeks', axis=1), on="Patient")

train_csv['WHERE'] = 'train'
test_csv['WHERE'] = 'val'
sub_csv['WHERE'] = 'test'
data = train_csv.append([sub_csv, test_csv])

In [10]:
columns = data.keys()
columns = list(columns)
print(columns)

print(train_csv.shape, test_csv.shape, sub_csv.shape, data.shape)
print(train_csv.Patient.nunique(), test_csv.Patient.nunique(), sub_csv.Patient.nunique(), 
      data.Patient.nunique())

['Patient', 'Weeks', 'FVC', 'Percent', 'Age', 'Sex', 'SmokingStatus', 'WHERE', 'Confidence', 'Patient_Week']
(1542, 8) (5, 8) (730, 10) (2277, 10)
176 5 5 176


In [11]:
data['min_week'] = data['Weeks']
data.loc[data.WHERE=='test','min_week'] = np.nan
data['min_week'] = data.groupby('Patient')['min_week'].transform('min')

base = data.loc[data.Weeks == data.min_week]
base = base[['Patient','FVC']].copy()
base.columns = ['Patient','min_FVC']
base['nb'] = 1
base['nb'] = base.groupby('Patient')['nb'].transform('cumsum')
base = base[base.nb==1]
base.drop('nb', axis=1, inplace=True)

data = data.merge(base, on='Patient', how='left')
data['base_week'] = data['Weeks'] - data['min_week']
del base

In [12]:
COLS = ['Sex','SmokingStatus']
FE = []
for col in COLS:
    for mod in data[col].unique():
        FE.append(mod)
        data[mod] = (data[col] == mod).astype(int)

In [14]:
data['age'] = (data['Age'] - data['Age'].min() ) / ( data['Age'].max() - data['Age'].min() )
data['BASE'] = (data['min_FVC'] - data['min_FVC'].min() ) / ( data['min_FVC'].max() - data['min_FVC'].min() )
data['week'] = (data['base_week'] - data['base_week'].min() ) / ( data['base_week'].max() - data['base_week'].min() )
data['percent'] = (data['Percent'] - data['Percent'].min() ) / ( data['Percent'].max() - data['Percent'].min() )
FE += ['age','percent','week','BASE']
data.head()

,Patient,Weeks,FVC,Percent,Age,Sex,SmokingStatus,WHERE,Confidence,Patient_Week,...,base_week,Male,Female,Ex-smoker,Never smoked,Currently smokes,age,BASE,week,percent
0,ID00007637202177411956430,-4,2315,58.253649,79,Male,Ex-smoker,train,NaN,NaN,...,0.0,1,0,1,0,0,0.769231,0.241456,0.179012,0.236393
1,ID00007637202177411956430,5,2214,55.712129,79,Male,Ex-smoker,train,NaN,NaN,...,9.0,1,0,1,0,0,0.769231,0.241456,0.234568,0.215941
2,ID00007637202177411956430,7,2061,51.862104,79,Male,Ex-smoker,train,NaN,NaN,...,11.0,1,0,1,0,0,0.769231,0.241456,0.246914,0.184960
3,ID00007637202177411956430,9,2144,53.950679,79,Male,Ex-smoker,train,NaN,NaN,...,13.0,1,0,1,0,0,0.769231,0.241456,0.259259,0.201767
4,ID00007637202177411956430,11,2069,52.063412,79,Male,Ex-smoker,train,NaN,NaN,...,15.0,1,0,1,0,0,0.769231,0.241456,0.271605,0.186580


In [15]:
patient_df = data.loc[data.WHERE=='train'].reset_index()
#chunk = data.loc[data.WHERE=='val']
#sub = data.loc[data.WHERE=='test']
del data

columns = patient_df.keys()
columns = list(columns)
print(columns)
patient_df.head()

['index', 'Patient', 'Weeks', 'FVC', 'Percent', 'Age', 'Sex', 'SmokingStatus', 'WHERE', 'Confidence', 'Patient_Week', 'min_week', 'min_FVC', 'base_week', 'Male', 'Female', 'Ex-smoker', 'Never smoked', 'Currently smokes', 'age', 'BASE', 'week', 'percent']


,index,Patient,Weeks,FVC,Percent,Age,Sex,SmokingStatus,WHERE,Confidence,...,base_week,Male,Female,Ex-smoker,Never smoked,Currently smokes,age,BASE,week,percent
0,0,ID00007637202177411956430,-4,2315,58.253649,79,Male,Ex-smoker,train,NaN,...,0.0,1,0,1,0,0,0.769231,0.241456,0.179012,0.236393
1,1,ID00007637202177411956430,5,2214,55.712129,79,Male,Ex-smoker,train,NaN,...,9.0,1,0,1,0,0,0.769231,0.241456,0.234568,0.215941
2,2,ID00007637202177411956430,7,2061,51.862104,79,Male,Ex-smoker,train,NaN,...,11.0,1,0,1,0,0,0.769231,0.241456,0.246914,0.184960
3,3,ID00007637202177411956430,9,2144,53.950679,79,Male,Ex-smoker,train,NaN,...,13.0,1,0,1,0,0,0.769231,0.241456,0.259259,0.201767
4,4,ID00007637202177411956430,11,2069,52.063412,79,Male,Ex-smoker,train,NaN,...,15.0,1,0,1,0,0,0.769231,0.241456,0.271605,0.186580


In [16]:
def load_scan(path):
    slices = [pydicom.read_file(path + '/' + s) for s in os.listdir(path)]
    slices.sort(key = lambda x: float(x.InstanceNumber))
    try:
        slice_thickness = np.abs(slices[0].ImagePositionPatient[2] - slices[1].ImagePositionPatient[2])
    except NameError:
        slice_thickness = np.abs(slices[0].SliceLocation - slices[1].SliceLocation)
    except:
        slice_thickness = slices[0].SliceThickness
        
    if slice_thickness==0:
            slice_thickness=slices[0].SliceThickness
    for s in slices:
        s.SliceThickness = slice_thickness
    
    return slices

In [17]:
def get_pixels_hu(slices):
    image = np.stack([s.pixel_array for s in slices])
    # Convert to int16
    image = image.astype(np.int16)
    # Set outside-of-scan pixels to 0
    # The intercept is usually -1024, so air is approximately 0
    image[image == -2000] = 0
    # Convert to Hounsfield units (HU)
    for slice_number in range(len(slices)):
        intercept = slices[slice_number].RescaleIntercept
        slope = slices[slice_number].RescaleSlope
        if slope != 1:
            image[slice_number] = slope * image[slice_number].astype(np.float64)
            image[slice_number] = image[slice_number].astype(np.int16)
        image[slice_number] += np.int16(intercept)
    return np.array(image, dtype=np.int16)

def window_image(image, window_center, window_width):
    img_min = window_center - window_width // 2
    img_max = window_center + window_width // 2
    window_image = image.copy()
    window_image[window_image < img_min] = img_min
    window_image[window_image > img_max] = img_max
    return window_image

In [18]:
def resample(image, scan, new_spacing=[1,1,1]):
    # Determine current pixel spacing
    spacing = np.array([scan[0].SliceThickness] + list(scan[0].PixelSpacing), dtype=np.float32)

    resize_factor = spacing / new_spacing
    new_real_shape = image.shape * resize_factor
    new_shape = np.round(new_real_shape)
    real_resize_factor = new_shape / image.shape
    new_spacing = spacing / real_resize_factor
    
    image = scipy.ndimage.interpolation.zoom(image, real_resize_factor, mode='nearest')
    
    return image, new_spacing

In [19]:
def get_segmented_lungs(im, threshold):
    '''
    Step 1: Convert into a binary image. 
    '''
    binary = np.array(im < threshold, dtype=np.int8)
    '''
    Step 2: Remove the blobs connected to the border of the image.
    '''
    cleared = clear_border(binary)
    '''
    Step 3: Label the image.
    '''
    label_image = label(cleared)
    '''
    Step 4: Keep the labels with 2 largest areas.
    '''
    areas = [r.area for r in regionprops(label_image)]
    areas.sort()
    if len(areas) > 2:
        for region in regionprops(label_image):
            if region.area < areas[-2]:
                for coordinates in region.coords:                
                       label_image[coordinates[0], coordinates[1]] = 0
    binary = label_image > 0
    '''
    Step 5: Erosion operation with a disk of radius 2. This operation is 
    seperate the lung nodules attached to the blood vessels.
    '''
    selem = disk(2)
    binary = binary_erosion(binary, selem)
    '''
    Step 6: Closure operation with a disk of radius 10. This operation is 
    to keep nodules attached to the lung wall.
    '''
    selem = disk(10)
    binary = binary_closing(binary, selem)
    '''
    Step 7: Fill in the small holes inside the binary mask of lungs.
    '''
    edges = roberts(binary)
    binary = ndi.binary_fill_holes(edges)
    '''
    Step 8: Superimpose the binary mask on the input image.
    '''
#     get_high_vals = binary == 0
#     im[get_high_vals] = 0
    im = binary* im
        
    return im, binary.astype(int)

In [20]:
#MIN_BOUND = -1000.0
#MAX_BOUND = 320.0
    
def normalize(image, MIN_BOUND, MAX_BOUND):
    image = (image - MIN_BOUND) / (MAX_BOUND - MIN_BOUND)
    image[image>1] = 1.
    image[image<0] = 0.
    return image

def lung_volume(masks, spacing):
    slice_thickness = spacing[0]
    pixel_spacing = (spacing[1], spacing[2])
    
    return np.round(np.sum(masks) * slice_thickness * pixel_spacing[0]*pixel_spacing[1], 3)

def lung_process(image, spacing, threshold):
    segmented = []
    masks = []
    for im in image:
        segment,mask = get_segmented_lungs(im,threshold)
        masks.append(mask.astype(int))
        segmented.append(segment)
    #vol = lung_volume(np.asarray(masks), spacing)
    return np.asarray(segmented), np.asarray(masks)

def compute_stats(img):
    kurt = kurtosis(img.ravel()[img.ravel() <0.6])
    ske = skew(img.ravel()[img.ravel() <0.6])

    std_i = img.ravel()[img.ravel() <0.6].std()
    mean_i = img.ravel()[img.ravel() <0.6].mean()
    median_i = np.median(img.ravel()[img.ravel() <0.6])
    return kurt, ske, std_i, mean_i, median_i

In [23]:
def preprocess_file(patient_id):
    patient = load_scan(INPUT_FOLDER + patient_id)
    patient_pixels = get_pixels_hu(patient)
    
    if patient_pixels.mean()<-1500 and patient_pixels.mean()>=-1800:
        lung_image = window_image(patient_pixels, -1500, 3000)
        pix_resampled, spacing = resample(lung_image, patient, [1,1,1])
        segmented, mask = lung_process(pix_resampled, spacing, -1400)
        normalized = normalize(segmented, -3000, 1500)
        
    elif patient_pixels.mean()<-1800:
        lung_image = window_image(patient_pixels, -3000, 4500)
        pix_resampled, spacing = resample(lung_image, patient, [1,1,1])
        segmented, mask = lung_process(pix_resampled, spacing, -2200)
        normalized = normalize(segmented, -4000, 300)
        
    else:
        lung_image = window_image(patient_pixels, -300, 1200)
        pix_resampled, spacing = resample(lung_image, patient, [1,1,1])
        segmented, mask = lung_process(pix_resampled, spacing, -200)
        normalized = normalize(segmented, -1500, 900)
        
    return normalized.astype(np.float16), mask

In [24]:
def chunks(l, n):
    # Credit: Ned Batchelder
    # Link: http://stackoverflow.com/questions/312443/how-do-you-split-a-list-into-evenly-sized-chunks
    """Yield successive n-sized chunks from l."""
    for i in range(0, len(l), n):
        yield l[i:i + n]

def mean(l):
    return sum(l) / len(l)

def reduce_slices(slices):
    new_slices = []
    chunk_sizes = math.ceil(len(slices) / HM_SLICES)
    for slice_chunk in chunks(slices, chunk_sizes):
        slice_chunk = list(map(mean, zip(*slice_chunk)))
        new_slices.append(slice_chunk)

    if len(new_slices) == HM_SLICES-1:
        new_slices.append(new_slices[-1])

    if len(new_slices) == HM_SLICES-2:
        new_slices.append(new_slices[-1])
        new_slices.append(new_slices[-1])

    if len(new_slices) == HM_SLICES+2:
        new_val = list(map(mean, zip(*[new_slices[HM_SLICES-1],new_slices[HM_SLICES],])))
        del new_slices[HM_SLICES]
        new_slices[HM_SLICES-1] = new_val

    if len(new_slices) == HM_SLICES+1:
        new_val = list(map(mean, zip(*[new_slices[HM_SLICES-1],new_slices[HM_SLICES],])))
        del new_slices[HM_SLICES]
        new_slices[HM_SLICES-1] = new_val
    return new_slices

In [25]:
save_img = dicom_arrays_dir
save_mask = mask_dir
def save_arrays(patient_ids):
    segmented, mask = preprocess_file(patient_ids)
    array_path = f'{save_img}/{patient_ids}.npy'
    mask_path = f'{save_mask}/{patient_ids}_mask.npy'
    
    np.save(str(array_path), segmented)
    np.save(str(mask_path), mask)
    gc.collect()

def cache_dataset():
    patient_ids = patient_df.drop_duplicates(subset=['Patient']).Patient

    with Pool(processes=4) as pool:
        show_run_results = list(
            tqdm(pool.imap(save_arrays, patient_ids), total = len(patient_ids))
        )

In [26]:
if volume_array_file.exists() and kurts_array_file.exists():
    print('loading pre-calculated arrays')
    volumes = torch.load(volume_array_file)
    kurts = torch.load(kurts_array_file)
    skews = torch.load(skews_array_file)
    means = torch.load(means_array_file)
    stds = torch.load(stds_array_file)
    medians = torch.load(medians_array_file)
    
    temp_df= patient_df.copy().drop_duplicates(subset=['Patient'])
else:
    print('Process dicom images and caching dataset...')
    volumes = []
    kurts = []
    skews = []
    means = []
    stds = []
    medians = []
    
    cache_dataset()
    
    temp_df= patient_df.copy().drop_duplicates(subset=['Patient'])
    print('Calculating image statistics...')
    
    for i, patient_id in tqdm(enumerate(temp_df.Patient), total=len(temp_df.Patient)):
        segmented = []
        cached_img_path = f'{dicom_arrays_dir}/{patient_id}.npy'
        cached_mask_file = mask_dir/f'{patient_id}_mask.npy'

        img_array = np.load(cached_img_path)
        mask = np.load(cached_mask_file)
        
        vol = lung_volume(np.asarray(mask), (1,1,1))
        kurt, ske, std_i, mean_i, median_i = compute_stats(img_array)
        
        volumes.append(vol)
        means.append(mean_i)
        stds.append(std_i)
        medians.append(median_i)
        kurts.append(kurt)
        skews.append(ske)
        
        gc.collect()

    torch.save(volumes, 'volume_array.pt')
    torch.save(kurts, 'kurts_array.pt')
    torch.save(skews, 'skews_array.pt')
    torch.save(means, 'mean_array.pt')
    torch.save(stds, 'std_array.pt')
    torch.save(medians, 'median_array.pt')

loading pre-calculated arrays


In [27]:
temp_df["volume"] = np.asarray(volumes)/1e6
temp_df["kurts"] = kurts
temp_df["skews"] = skews
temp_df["mean_vals"] = means
#temp_df["std_vals"] = stds
#temp_df["median_vals"] = medians
temp_df.head()

,index,Patient,Weeks,FVC,Percent,Age,Sex,SmokingStatus,WHERE,Confidence,...,Never smoked,Currently smokes,age,BASE,week,percent,volume,kurts,skews,mean_vals
0,0,ID00007637202177411956430,-4,2315,58.253649,79,Male,Ex-smoker,train,NaN,...,0,0,0.769231,0.241456,0.179012,0.236393,3.515550,0.742115,1.192383,0.326172
9,9,ID00009637202177434476278,8,3660,85.282878,69,Male,Ex-smoker,train,NaN,...,0,0,0.512821,0.491270,0.179012,0.453901,4.552959,2.920642,1.780273,0.300293
18,18,ID00010637202177584971671,0,3523,94.724672,60,Male,Ex-smoker,train,NaN,...,0,0,0.282051,0.465825,0.179012,0.529881,2.481575,0.693160,1.090820,0.337402
27,27,ID00011637202177653955184,6,3326,85.987590,72,Male,Ex-smoker,train,NaN,...,0,0,0.589744,0.429235,0.179012,0.459572,4.649744,1.347476,1.318359,0.312500
36,36,ID00012637202177665765362,33,3418,93.726006,65,Male,Never smoked,train,NaN,...,1,0,0.410256,0.446322,0.179012,0.521844,4.414031,2.529640,1.617188,0.303711


In [28]:
patient_df=patient_df.merge(temp_df[['Patient','volume','kurts','skews','mean_vals']],how='left',on='Patient')
print(len(patient_df))
patient_df.head()

1542


,index,Patient,Weeks,FVC,Percent,Age,Sex,SmokingStatus,WHERE,Confidence,...,Never smoked,Currently smokes,age,BASE,week,percent,volume,kurts,skews,mean_vals
0,0,ID00007637202177411956430,-4,2315,58.253649,79,Male,Ex-smoker,train,NaN,...,0,0,0.769231,0.241456,0.179012,0.236393,3.51555,0.742115,1.192383,0.326172
1,1,ID00007637202177411956430,5,2214,55.712129,79,Male,Ex-smoker,train,NaN,...,0,0,0.769231,0.241456,0.234568,0.215941,3.51555,0.742115,1.192383,0.326172
2,2,ID00007637202177411956430,7,2061,51.862104,79,Male,Ex-smoker,train,NaN,...,0,0,0.769231,0.241456,0.246914,0.184960,3.51555,0.742115,1.192383,0.326172
3,3,ID00007637202177411956430,9,2144,53.950679,79,Male,Ex-smoker,train,NaN,...,0,0,0.769231,0.241456,0.259259,0.201767,3.51555,0.742115,1.192383,0.326172
4,4,ID00007637202177411956430,11,2069,52.063412,79,Male,Ex-smoker,train,NaN,...,0,0,0.769231,0.241456,0.271605,0.186580,3.51555,0.742115,1.192383,0.326172


In [29]:
#patient_df['skews'].isnull().values.any()
df1 = patient_df[patient_df.isna().any(axis=1)]
df1.head(20).drop_duplicates('Patient')

,index,Patient,Weeks,FVC,Percent,Age,Sex,SmokingStatus,WHERE,Confidence,...,Never smoked,Currently smokes,age,BASE,week,percent,volume,kurts,skews,mean_vals
0,0,ID00007637202177411956430,-4,2315,58.253649,79,Male,Ex-smoker,train,NaN,...,0,0,0.769231,0.241456,0.179012,0.236393,3.515550,0.742115,1.192383,0.326172
9,9,ID00009637202177434476278,8,3660,85.282878,69,Male,Ex-smoker,train,NaN,...,0,0,0.512821,0.491270,0.179012,0.453901,4.552959,2.920642,1.780273,0.300293
18,18,ID00010637202177584971671,0,3523,94.724672,60,Male,Ex-smoker,train,NaN,...,0,0,0.282051,0.465825,0.179012,0.529881,2.481575,0.693160,1.090820,0.337402


In [30]:
patient_df['kurts'].fillna((patient_df['kurts'].mean()), inplace=True)
patient_df['skews'].fillna((patient_df['skews'].mean()), inplace=True)
patient_df['mean_vals'].fillna((patient_df['mean_vals'].mean()), inplace=True)
#patient_df['median_vals'].fillna((patient_df['median_vals'].mean()), inplace=True)
#patient_df['std_vals'].fillna((patient_df['std_vals'].mean()), inplace=True)

FE += ['kurts','skews','mean_vals']
print(FE)

['Male', 'Female', 'Ex-smoker', 'Never smoked', 'Currently smokes', 'age', 'percent', 'week', 'BASE', 'kurts', 'skews', 'mean_vals']


In [31]:
class AutoEncoder(nn.Module):
    def __init__(self, latent_features=10):
        super(AutoEncoder, self).__init__()
        # Encoder
        self.conv1 = nn.Conv3d(1, 16, 3)
        self.conv2 = nn.Conv3d(16, 32, 3)
        self.conv3 = nn.Conv3d(32, 96, 2)
        self.conv4 = nn.Conv3d(96, 1, 1)
        self.pool1 = nn.MaxPool3d(kernel_size=2, stride=2, return_indices=True)
        self.pool2 = nn.MaxPool3d(kernel_size=3, stride=3, return_indices=True)
        self.pool3 = nn.MaxPool3d(kernel_size=2, stride=2, return_indices=True)
        self.pool4 = nn.MaxPool3d(kernel_size=2, stride=2, return_indices=True)
        self.fc1 = nn.Linear(10 * 10, latent_features)
        # Decoder
        self.fc2 = nn.Linear(latent_features, 10 * 10)
        self.deconv0 = nn.ConvTranspose3d(1, 96, 1)
        self.deconv1 = nn.ConvTranspose3d(96, 32, 2)
        self.deconv2 = nn.ConvTranspose3d(32, 16, 3)
        self.deconv3 = nn.ConvTranspose3d(16, 1, 3)
        self.unpool0 = nn.MaxUnpool3d(kernel_size=2, stride=2)
        self.unpool1 = nn.MaxUnpool3d(kernel_size=2, stride=2)
        self.unpool2 = nn.MaxUnpool3d(kernel_size=3, stride=3)
        self.unpool3 = nn.MaxUnpool3d(kernel_size=2, stride=2)

    def encode(self, x, return_partials=True):
        # Encoder
        x = self.conv1(x)
        up3out_shape = x.shape
        x, i1 = self.pool1(x)

        x = self.conv2(x)
        up2out_shape = x.shape
        x, i2 = self.pool2(x)

        x = self.conv3(x)
        up1out_shape = x.shape
        x, i3 = self.pool3(x)

        x = self.conv4(x)
        up0out_shape = x.shape
        x, i4 = self.pool4(x)

        x = x.view(-1, 10 * 10)
        x = F.relu(self.fc1(x))

        if return_partials:
            return x, up3out_shape, i1, up2out_shape, i2, up1out_shape, i3, \
                   up0out_shape, i4

        else:
            return x

    def forward(self, x):
        x, up3out_shape, i1, up2out_shape, i2, \
        up1out_shape, i3, up0out_shape, i4 = self.encode(x)

        # Decoder
        x = F.relu(self.fc2(x))
        x = x.view(-1, 1, 1, 10, 10)
        x = self.unpool0(x, output_size=up0out_shape, indices=i4)
        x = self.deconv0(x)
        x = self.unpool1(x, output_size=up1out_shape, indices=i3)
        x = self.deconv1(x)
        x = self.unpool2(x, output_size=up2out_shape, indices=i2)
        x = self.deconv2(x)
        x = self.unpool3(x, output_size=up3out_shape, indices=i1)
        x = self.deconv3(x)

        return x

In [36]:
gkf = GroupKFold(n_splits=5)
groups = patient_df['Patient']
patient_df['fold'] = -1
for i, (train_idx, valid_idx) in enumerate(gkf.split(patient_df, patient_df['FVC'], groups)):
    patient_df.loc[valid_idx, 'fold'] = i

In [37]:
# From https://github.com/Bjarten/early-stopping-pytorch
class EarlyStopping:
    """Early stops the training if validation loss doesn't improve after a given patience."""
    def __init__(self, patience=7, verbose=False):
        """
        Args:
            patience (int): How long to wait after last time validation loss improved.
                            Default: 7
            verbose (bool): If True, prints a message for each validation loss improvement. 
                            Default: False
        """
        self.patience = patience
        self.verbose = verbose
        self.counter = 0
        self.best_score = None
        self.early_stop = False
        self.val_loss_min = np.Inf

    def __call__(self, val_loss, model):

        #score = -val_loss
        score = val_loss

        if self.best_score is None:
            self.best_score = score
            #self.save_checkpoint(val_loss, model)
        elif score > self.best_score:
            self.counter += 1
            print(f'EarlyStopping counter: {self.counter} out of {self.patience}')
            if self.counter >= self.patience:
                self.early_stop = True
        else:
            self.best_score = score
            #self.save_checkpoint(val_loss, model)
            self.counter = 0

    def save_checkpoint(self, val_loss, model):
        '''Saves model when validation loss decrease.'''
        if self.verbose:
            print(f'Validation loss decreased ({self.val_loss_min:.6f} --> {val_loss:.6f}).  Saving model ...')
        torch.save(model.state_dict(), 'checkpoint.pt')
        self.val_loss_min = val_loss

In [38]:
autoencoder_models = []
for path in MODELS:
    state_dict = torch.load(path,map_location=torch.device('cpu'))
    model = AutoEncoder()
    model.load_state_dict(state_dict)
    model.to(device)
    model.float()
    model.eval()
    autoencoder_models.append(model)

In [39]:
# Helper function that generates all latent features
class GenerateLatentFeatures:
    def __init__(self, autoencoder_models, latent_dir):
        #self.df = df.drop_duplicates(subset=['Patient'])
        self.latent_dir = Path(latent_dir)
        #self.cache_dir = Path(cache_dir)
        
    def __call__(self, img_id, img_array):
        patient_id = img_id
        cached_latent_file = self.latent_dir/f'{img_id}_lat.pt'

        if cached_latent_file.is_file():
            latent_features = torch.load(cached_latent_file, map_location=torch.device('cpu'))
        else:
            latent_features = []

            if len(img_array)>HM_SLICES:
                img_array = np.asarray(reduce_slices(img_array))
                if len(img_array) < HM_SLICES:
                   img_array = np.pad(img_array,[[0,HM_SLICES-len(img_array)],[0,0],[0,0]],constant_values=0.0)
            else:
                if len(img_array) < HM_SLICES:
                   img_array = np.pad(img_array,[[0,HM_SLICES-len(img_array)],[0,0],[0,0]],constant_values=0.0)

            img = torch.tensor(img_array).unsqueeze(0).float()
            img = F.interpolate(img, size=256)
            img = img.view(img.shape[0], 1, img.shape[1], img.shape[2], img.shape[3])
            img = torch.tensor(img).to(device)

            preds = 0.0
            with torch.no_grad():
                for model in autoencoder_models:
                    pred = model.encode(img, return_partials=False).squeeze(0)
                    preds+=pred.detach().cpu().numpy()
                preds = preds/len(autoencoder_models)
            latent_features.append(preds)

            latent_features = np.concatenate(latent_features)
            torch.save(latent_features, cached_latent_file)
            
        return latent_features

In [40]:
class fibrosisDataset(Dataset):
    def __init__(self,
                 df,
                 rand=False,
                 mode='train',
                 extract_features=None,
                ):
        self.df = df.sort_values(by=['Patient','Weeks'],ascending=True).reset_index(drop=True)
        self.rand = rand
        self.mode = mode
        self.extract_features = extract_features

    def __len__(self):
        return len(self.df)

    def __getitem__(self, index):
        row = self.df.iloc[index]
        patient_id = row.Patient
        label = row.FVC

        tabular_data = row[FE]
        
        file_path = f'../input/dicom-arrays-processed/kaggle/dicom_arrays/{patient_id}.npy'
        img_array = np.load(file_path)
        
        if self.extract_features:
            features = self.extract_features(patient_id, img_array)
        
        if self.mode=='train' or self.mode=='valid':
            return torch.tensor(tabular_data), torch.tensor(label), torch.tensor(features)
        else:
            return torch.tensor(tabular_data)

In [41]:
patient_df.head()

,index,Patient,Weeks,FVC,Percent,Age,Sex,SmokingStatus,WHERE,Confidence,...,Currently smokes,age,BASE,week,percent,volume,kurts,skews,mean_vals,fold
0,0,ID00007637202177411956430,-4,2315,58.253649,79,Male,Ex-smoker,train,NaN,...,0,0.769231,0.241456,0.179012,0.236393,3.51555,0.742115,1.192383,0.326172,0
1,1,ID00007637202177411956430,5,2214,55.712129,79,Male,Ex-smoker,train,NaN,...,0,0.769231,0.241456,0.234568,0.215941,3.51555,0.742115,1.192383,0.326172,0
2,2,ID00007637202177411956430,7,2061,51.862104,79,Male,Ex-smoker,train,NaN,...,0,0.769231,0.241456,0.246914,0.184960,3.51555,0.742115,1.192383,0.326172,0
3,3,ID00007637202177411956430,9,2144,53.950679,79,Male,Ex-smoker,train,NaN,...,0,0.769231,0.241456,0.259259,0.201767,3.51555,0.742115,1.192383,0.326172,0
4,4,ID00007637202177411956430,11,2069,52.063412,79,Male,Ex-smoker,train,NaN,...,0,0.769231,0.241456,0.271605,0.186580,3.51555,0.742115,1.192383,0.326172,0


In [42]:
train_dataset = fibrosisDataset(patient_df, mode='train', extract_features=GenerateLatentFeatures(autoencoder_models, latent_dir))
train_loader = DataLoader(train_dataset, batch_size=8, shuffle=False, num_workers=0)

for (tabular_data,label, features) in train_loader:
    inputs = tabular_data
    print(tabular_data.shape)
    print(label)
    print(features.shape)
    break

torch.Size([8, 12])
tensor([2315, 2214, 2061, 2144, 2069, 2101, 2000, 2064])
torch.Size([8, 10])


In [49]:
class RNNFeatures(nn.Module):    
    def __init__(self, input_dim, hidden_dim, layer_dim, output_dim, in_ctscan_features=10):
        super().__init__()
        self.hidden_dim = hidden_dim
        self.layer_dim = layer_dim
        self.in_ctscan_features = in_ctscan_features
        self.match_sz = nn.Linear(in_ctscan_features, input_dim)
        
        self.rnn = nn.RNN(input_dim*2, hidden_dim, layer_dim, batch_first=True, nonlinearity='relu',dropout=0.1)
        self.fc = nn.Linear(hidden_dim, hidden_dim)
        self.fc_out = nn.Linear(hidden_dim, output_dim)
        #self.batch_size = None
        #self.hidden = None
    
    def forward(self, x1, x2):
        x1 = x1.view(-1, len(x1), len(x1[0]))
        x2 = F.relu(self.match_sz(x2))
        x2 = x2.view(-1, len(x2), len(x2[0]))
        
        x = torch.cat([x1, x2], dim=2)
        
        h0 = self.init_hidden(x)
        out, hn = self.rnn(x, h0)
        out = F.relu(self.fc(out[:, -1, :]))
        out = self.fc_out(out)
        return out
    
    def init_hidden(self, x):
        h0 = torch.zeros(self.layer_dim, x.size(0), self.hidden_dim)
        return h0

In [51]:
def metric(preds, targets):
    sigma = preds[:, 2] - preds[:, 0]
    sigma[sigma < 70] = 70
    delta = (preds[:, 1] - targets).abs()
    delta[delta > 1000] = 1000
    return (-np.sqrt(2) * delta / sigma - torch.log(np.sqrt(2) * sigma)).mean()

def fvc_loss(pred_fvc,true_fvc):
    true_fvc=torch.reshape(true_fvc,pred_fvc[:,1].shape)
    fvc_err=torch.abs(pred_fvc-true_fvc)
    return fvc_err

def quantile_loss(preds, target, quantiles):
    assert not target.requires_grad
    assert preds.size(0) == target.size(0)
    losses = []
    for i, q in enumerate(quantiles):
        errors = target - preds[:, i]
        losses.append(torch.max((q - 1)*errors, q*errors).unsqueeze(1))
    loss = torch.mean(torch.sum(torch.cat(losses, dim=1), dim=1))
    return loss

def mloss(y_pred, y_true, _lambda):
    return _lambda * quantile_loss(y_pred, y_true, quantiles) + (1 - _lambda)*metric(y_pred, y_true)


In [52]:
def train(epoch):
    model.train()
    train_loss = []
    PREDS = []
    TARGETS = []
    optimizer.zero_grad()
    running_score = 0.0
    
    bar = tqdm(enumerate(train_loader), total=len(train_loader))
    for steps, (tabular_data, label, features) in bar:
        
        optimizer.zero_grad()
        
        tabular_data =  tabular_data.to(device).float()
        label = label.to(device).float()
        
        preds = model(tabular_data, features)
        loss = quantile_loss(preds, label, quantiles)
        #loss = loss/accumulation_steps
        
        if fp16:
            with amp.scale_loss(loss, optimizer) as scaled_loss:
                scaled_loss.backward()
        else:
            loss.backward()
            
#         if steps % accumulation_steps == 0:
#             optimizer.step()
#             scheduler.step()
#             optimizer.zero_grad()
        
        optimizer.step()
        
        running_score += metric(preds, label)
        
        PREDS.append(preds.detach())
        TARGETS.append(label)
        loss_np = loss.detach().cpu().numpy()
        train_loss.append(loss_np)
        smooth_loss = sum(train_loss[-100:]) / min(len(train_loss), 100)
        neptune.send_metric('lr_iter', optimizer.param_groups[0]['lr'])
    
    PREDS = torch.cat(PREDS).cpu().numpy()
    TARGETS = torch.cat(TARGETS).cpu().numpy()   
    score = running_score/len(train_loader)
    train_loss = np.mean(train_loss)
    
    neptune.send_metric('train_loss', train_loss)
    neptune.send_metric('train_score', score)
    neptune.send_metric('lr', optimizer.param_groups[0]['lr'])
    
    return train_loss, score

In [53]:
def valid(epoch):
    model.eval()
    valid_loss = []
    PREDS = []
    TARGETS = []
    running_score = 0.0
    
    bar = tqdm(enumerate(valid_loader), total=len(valid_loader))
    with torch.no_grad():
        for steps, (tabular_data, label, features) in bar:

            tabular_data =  tabular_data.to(device).float()
            label = label.to(device).float()

            preds = model(tabular_data, features)
            loss = quantile_loss(preds, label, quantiles)
            
            running_score += metric(preds, label)

            PREDS.append(preds.detach())
            TARGETS.append(label)
            loss_np = loss.detach().cpu().numpy()
            valid_loss.append(loss_np)
    
    PREDS = torch.cat(PREDS).cpu().numpy()
    TARGETS = torch.cat(TARGETS).cpu().numpy()   
    score = running_score/len(valid_loader)
    valid_loss = np.mean(valid_loss)
    sigma_opt = mean_absolute_error(TARGETS, PREDS[:, 1])
    unc = PREDS[:,2] - PREDS[:, 0]
    sigma_mean = np.mean(unc)
        
    neptune.send_metric('valid_loss', valid_loss)
    neptune.send_metric('valid_score', score)
    neptune.send_metric('sigma_opt', sigma_opt)
    neptune.send_metric('sigma_mean', sigma_mean)
    
    return valid_loss, score, PREDS, TARGETS

In [54]:
def run_main(epochs, fold):
    best_score = -np.inf
    best_loss = np.inf
    best_preds = None
    for epoch in range(1, epochs+1):
        start_time = time.time()
        torch.cuda.empty_cache()
        gc.collect()
        scheduler.step(epoch-1)
        avg_train_loss, train_score = train(epoch) 
        avg_val_loss, valid_score, oof_pred, oof_target = valid(epoch)
        
        elapsed = time.time() - start_time
        LOGGER.debug(f'  Epoch {epoch} - avg_train_loss: {avg_train_loss:.4f}  avg_val_loss: {avg_val_loss:.4f}  time: {elapsed:.0f}s')
        LOGGER.debug(f'  Epoch {epoch} - train_metric: {train_score}  valid_metric: {valid_score}')
        if valid_score>best_score:
            torch.save(model.state_dict(), f'model_fold_{fold}.pt')
            best_score=valid_score
            oof_csv = pd.DataFrame(data=oof_pred, columns=list(quantiles))
            oof_csv['oof_target'] = oof_target
            oof_csv.to_csv(f'oof_fold_{fold}.csv', index=False)
        checkpoint = {
            'epoch': epoch,
            'lr': optimizer.param_groups[0]['lr'],
            'state_dict': model.state_dict(),
            'optimizer': optimizer.state_dict(),
            'scheduler_state_dict': scheduler.state_dict(),
            }
        torch.save(checkpoint, 'checkpoint.pt')
        
        early_stopping(avg_val_loss, model)
        if early_stopping.early_stop:
            print("Early stopping")
            break
        
    return valid_score

In [ ]:
valid_score = []
valid_labels = []
for fold in range(CFG.n_fold):
    print(f'Training fold {fold}')
    
    model = RNNFeatures(12, 150, 2, 3).to(device) 

    optimizer = torch.optim.Adam(model.parameters(), lr=CFG.lr)
    scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, CFG.epochs)

    train_idx = np.where((patient_df['fold'] != fold))[0]
    valid_idx = np.where((patient_df['fold'] == fold))[0]

    train_data  = patient_df.loc[train_idx]
    valid_data = patient_df.loc[valid_idx] 

    train_dataset = fibrosisDataset(train_data, mode='train',extract_features=GenerateLatentFeatures(autoencoder_models, latent_dir))
    train_loader = DataLoader(train_dataset, batch_size=CFG.batch_size, shuffle=True, num_workers=num_workers, pin_memory=True)

    valid_dataset = fibrosisDataset(valid_data, mode='train',extract_features=GenerateLatentFeatures(autoencoder_models, latent_dir))
    valid_loader = DataLoader(valid_dataset, batch_size=CFG.batch_size, shuffle=False, num_workers=0, pin_memory=True)
    
    early_stopping = EarlyStopping(patience=20, verbose=False)
    
    score = run_main(CFG.epochs, fold)
    valid_score.append(score)
    del train_dataset, train_loader, valid_dataset, valid_loader, model
    gc.collect()

In [56]:
oof = []
for fold in range(CFG.n_fold):
    oof_csv = pd.read_csv(f'./oof_fold_{fold}.csv')
    oof.append(oof_csv)
    
oof = np.concatenate(oof)
oof = np.transpose(oof)

d =  {'0.2': oof[0], '0.5': oof[1], '0.8': oof[2], 'oof_target': oof[3], 'Patient':train_csv.Patient, 'Weeks':train_csv.Weeks}
oof_preds = pd.DataFrame(data=d)
oof_preds.to_csv('oof.csv', index=False)
print(oof.shape)

unique_ids = oof_preds.Patient.drop_duplicates()
print(len(unique_ids))

oof_columns = oof_preds.keys()
oof_columns = list(oof_columns)

oof_last = pd.DataFrame(columns=oof_columns)
filtered = []
for patient_id in unique_ids:
    check = oof_preds.loc[oof_preds.Patient==str(patient_id)]
    largest = check.nlargest(columns='Weeks',n=3, keep='first')
    filtered.append(largest)

filtered = np.concatenate(filtered)
oof_last = pd.DataFrame(filtered, columns=oof_columns)
oof_last.to_csv('oof_last.csv')

(4, 1542)
176


In [57]:
oof_csv = pd.read_csv('oof.csv')
oof_predictions = torch.tensor(np.asarray(oof_csv[['0.2','0.5','0.8']]))
oof_targets = torch.tensor(oof_csv['oof_target'])

oof_last = pd.read_csv('oof_last.csv')
oof_last_predictions = torch.tensor(np.asarray(oof_last[['0.2','0.5','0.8']]))
oof_last_targets = torch.tensor(oof_last['oof_target'])

oof_score = metric(oof_predictions, oof_targets)
oof_last_score = metric(oof_last_predictions, oof_last_targets)
print("oof score: ",oof_score)
print("last 3 measurements score", oof_last_score)

oof score:  tensor(-6.7114, dtype=torch.float64)
last 3 measurements score tensor(-6.7109, dtype=torch.float64)


In [58]:
'''
Check against best scored kernel oof predictions
'''
oof_check = pd.read_csv(f'../input/oof-check/oof.csv')
oof_predictions1 = torch.tensor(np.asarray(oof_check[['0.2','0.5','0.8']]))
oof_targets1 = torch.tensor(oof_check['oof_target'])

oof_last3 = pd.read_csv(f'../input/oof-check/best_oof_last3.csv')
oof_last3_predictions = torch.tensor(np.asarray(oof_last3[['0.2','0.5','0.8']]))
oof_last3_targets = torch.tensor(oof_last3['oof_target'])

oof_score = metric(oof_predictions1, oof_targets1)
oof_last3_score = metric(oof_last3_predictions, oof_last3_targets)
print("best oof score: ",oof_score)
print("best last 3 measurements score", oof_last3_score)

best oof score:  tensor(-6.6694, dtype=torch.float64)
best last 3 measurements score tensor(-6.6752, dtype=torch.float64)


In [59]:
neptune.stop()